In [31]:
import pandas as pd
import json
import os
import re
import ast

pd.set_option('display.max_columns', None)

In [2]:
# customer repeat orders -customer_id

with open("iseller_api_response.json", "r") as file:
    data = json.load(file)

In [3]:
def normalize_list_dictionary(dictionary):
    normalized_dict_list = []
    for element in dictionary:
        try:
            num_orders = len(element)
            index = 0
            while num_orders > index:
                normalized_dict_list.append(element[index])
                index += 1
        except:
            pass
    # Remove Keys with None values
    filtered_orders_dict = [order for order in normalized_dict_list if order is not None]
    return filtered_orders_dict

In [4]:
def proccess_iseller_data(data):
    orders_df = pd.DataFrame(data['orders'])
    
    order_details_dict = orders_df["order_details"].to_list()
    normalized_order_details = normalize_list_dictionary(order_details_dict)
    order_details = pd.DataFrame(normalized_order_details)

    #rename columns, grand_total_amount refers to the entire amount of the whole order while total_order_amount refers to total product for a product type
    orders_df.rename(columns = {'total_order_amount':' grand_total_amount'}, inplace = True) 

    #merge columns
    relevant_order_details = order_details[["order_id", "product_id", "product_name", "product_type", "fulfillment_status", "quantity", "base_price", "total_order_amount", "cogs"]]
    merged_orders = orders_df.merge(relevant_order_details, how="left", on="order_id")
    
    # convert to correct dtypes
    merged_orders['order_date'] = pd.to_datetime(merged_orders['order_date'])
    merged_orders['closed_date'] = pd.to_datetime(merged_orders['closed_date'])

    return merged_orders


def get_list_json(input_folder):
    # Initialize a list to store all JSON data
    all_data = []

    # Iterate through each JSON file in the folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.json'):
            file_path = os.path.join(input_folder, filename)
            with open(file_path, 'r', encoding='utf-8') as json_file:
                data = json.load(json_file)
                if isinstance(data, list):
                    all_data.extend(data)
                elif isinstance(data, dict):
                    all_data.append(data)

    return all_data


In [5]:
def extract_volume(product_name):
    volume_regex = r'(\d+(\.\d+)?\s*(ml|l))'  # Regex pattern to match both 'ml' and 'l' formats
    #match = re.search(volume_regex, product_name)
    match = re.search(volume_regex, str(product_name))  # Ensure product_name is converted to string

    if match:
        return match.group(1)
    else:
        return 'unspecified'
    
def clean_product_name(product_name):
    # volume_regex = r'\b\d+(\.\d+)?\s*(ml|l)\b'  # '\b' for word boundaries
    # pattern = r'\b\d+(\.\d+)?\s*(ml|l)\b|bli - |gof - '  # '\b' for word boundaries
    pattern = r'\b\d+(\.\d+)?\s*(ml|l)\b|bli - |gof - |- resell ecer| - resell dus|-resell dus|- dus|cabang - '  # '\b' for word boundaries

    return re.sub(pattern, '', str(product_name))  # Ensure product_name is converted to string


In [6]:
# # Store list of json data 
list_json_data = get_list_json('data/raw_copy')
test_json = list_json_data[:5]
list_df = []

# concat list of df 
for i in list_json_data:
    df = proccess_iseller_data(i)
    list_df.append(df)

full_iseller_data = pd.concat(list_df)

In [7]:
full_iseller_data['order_date'] = pd.to_datetime(full_iseller_data['order_date'])
full_iseller_data['hour'] = full_iseller_data['order_date'].dt.hour
full_iseller_data['day'] = full_iseller_data['order_date'].dt.day
full_iseller_data['month'] = full_iseller_data['order_date'].dt.month
full_iseller_data['week'] = full_iseller_data['order_date'].dt.isocalendar().week
full_iseller_data['Quarter'] = full_iseller_data['order_date'].dt.quarter


# get location columns, get coordinates ?
full_iseller_data['location'] = full_iseller_data['outlet_name'].str.split(' - ').str[0]


# standardize naming
full_iseller_data['product_name'] = full_iseller_data['product_name'].str.lower()
full_iseller_data['Volume'] = full_iseller_data['product_name'].apply(extract_volume)
full_iseller_data['clean_product_name'] = full_iseller_data['product_name'].apply(clean_product_name)
full_iseller_data['clean_product_name'] = full_iseller_data['clean_product_name'].str.strip()


In [8]:
#full_iseller_data.to_csv('full_iseller_data.csv')

In [9]:
print("First Order: ", full_iseller_data["order_date"].min())
print("Latest Order: ", full_iseller_data["order_date"].max())

First Order:  2023-06-01 00:38:56
Latest Order:  2023-12-31 23:42:41


In [10]:
full_iseller_data.columns

Index(['external_customer_id', 'order_id', 'order_reference', 'channel',
       'channel_detail', 'marketplace_account', 'order_date', 'closed_date',
       'status', 'payment_status', 'fulfillment_status_x', 'notes',
       'processing_method', 'cancel_date', 'cancel_reason', 'outlet_id',
       'outlet_name', 'outlet_code', 'register_id', 'register_name',
       'customer_id', 'customer_first_name', 'customer_last_name',
       'customer_phone_number', 'customer_email', 'server_id',
       'server_first_name', 'server_last_name', 'cashier_id',
       'cashier_first_name', 'cashier_last_name', 'currency',
       ' grand_total_amount', 'discount_percentage', 'total_discount_amount',
       'total_promotion_amount', 'subtotal', 'total_tax_amount',
       'total_additional_final_amount', 'total_additional_subtotal_amount',
       'total_additional_order_amount', 'rounding_amount', 'total_amount',
       'buying_price', 'total_shipping_amount', 'total_weight',
       'tax_inclusive', 'on_

In [11]:
# anggur merah gold cap orang tua anggur merah premium	anggur kolesom 17.5% cap orang tua are these (are all amer cap orang tua)

In [12]:
# Filter rows where 'items_list' has more than two elements
filtered_df = full_iseller_data[full_iseller_data['order_details'].apply(lambda x: len(x) > 2)]

# Show the filtered DataFrame
filtered_df.head()

,external_customer_id,order_id,order_reference,channel,channel_detail,marketplace_account,order_date,closed_date,status,payment_status,fulfillment_status_x,notes,processing_method,cancel_date,cancel_reason,outlet_id,outlet_name,outlet_code,register_id,register_name,customer_id,customer_first_name,customer_last_name,customer_phone_number,customer_email,server_id,server_first_name,server_last_name,cashier_id,cashier_first_name,cashier_last_name,currency,grand_total_amount,discount_percentage,total_discount_amount,total_promotion_amount,subtotal,total_tax_amount,total_additional_final_amount,total_additional_subtotal_amount,total_additional_order_amount,rounding_amount,total_amount,buying_price,total_shipping_amount,total_weight,tax_inclusive,on_account,remaining_on_account,billing,shipping,device_location,order_type,order_details,promotions,discounts,transactions,partner_loyalties,product_id,product_name,product_type,fulfillment_status_y,quantity,base_price,total_order_amount,cogs,hour,day,month,week,Quarter,location,Volume,clean_product_name
10,None,5261ef57-dbc8-4fdf-a026-0d6f33f251e2,#152182,web,None,None,2023-06-28 16:50:26,2023-06-28 16:50:26,completed,paid,fulfilled,,checkout,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,435690eb-bb34-4801-b6aa-c5584af329ea,Register #100 - samsung SM-T505,92c62325-7678-45af-af04-bc5e65f289bd,M,suparna,+6281310307299,None,minumyukkaka.adm4@gmail.com,Surya,MYK Bekasi,None,None,None,IDR,446000.0,None,0.0,0.0,446000.0,0.0,0.0,0.0,0.0,0.0,468000.0,376100.0,18000.0,6380,False,False,0.0,None,{'address_id': '7beeff34-5deb-4426-96aa-2d45b5...,None,delivery,[{'order_id': '5261ef57-dbc8-4fdf-a026-0d6f33f...,[],[],[],None,04234a59-404d-46b6-aa45-ef90ea0c910d,mixmax anggur merah 275ml,VODKA,fulfilled,2.0,25000.0,50000.0,[{'product_id': '04234a59-404d-46b6-aa45-ef90e...,16,28,6,26,2,Bekasi,275ml,mixmax anggur merah
11,None,5261ef57-dbc8-4fdf-a026-0d6f33f251e2,#152182,web,None,None,2023-06-28 16:50:26,2023-06-28 16:50:26,completed,paid,fulfilled,,checkout,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,435690eb-bb34-4801-b6aa-c5584af329ea,Register #100 - samsung SM-T505,92c62325-7678-45af-af04-bc5e65f289bd,M,suparna,+6281310307299,None,minumyukkaka.adm4@gmail.com,Surya,MYK Bekasi,None,None,None,IDR,446000.0,None,0.0,0.0,446000.0,0.0,0.0,0.0,0.0,0.0,468000.0,376100.0,18000.0,6380,False,False,0.0,None,{'address_id': '7beeff34-5deb-4426-96aa-2d45b5...,None,delivery,[{'order_id': '5261ef57-dbc8-4fdf-a026-0d6f33f...,[],[],[],None,e717f7ad-3eef-4b69-bf51-6c4ad0d11bb2,mixmax exotic blue 275ml,VODKA,fulfilled,2.0,25000.0,50000.0,[{'product_id': 'e717f7ad-3eef-4b69-bf51-6c4ad...,16,28,6,26,2,Bekasi,275ml,mixmax exotic blue
12,None,5261ef57-dbc8-4fdf-a026-0d6f33f251e2,#152182,web,None,None,2023-06-28 16:50:26,2023-06-28 16:50:26,completed,paid,fulfilled,,checkout,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,435690eb-bb34-4801-b6aa-c5584af329ea,Register #100 - samsung SM-T505,92c62325-7678-45af-af04-bc5e65f289bd,M,suparna,+6281310307299,None,minumyukkaka.adm4@gmail.com,Surya,MYK Bekasi,None,None,None,IDR,446000.0,None,0.0,0.0,446000.0,0.0,0.0,0.0,0.0,0.0,468000.0,376100.0,18000.0,6380,False,False,0.0,None,{'address_id': '7beeff34-5deb-4426-96aa-2d45b5...,None,delivery,[{'order_id': '5261ef57-dbc8-4fdf-a026-0d6f33f...,[],[],[],None,9e61efee-f5e4-42cd-90cb-bd9518ae3e6a,iceland vodka orange 700ml,VODKA,fulfilled,2.0,173000.0,346000.0,[{'product_id': '9e61efee-f5e4-42cd-90cb-bd951...,16,28,6,26,2,Bekasi,700ml,iceland vodka orange
26,,e014d96a-7bac-42a3-b3c4-9d4237e078e3,#152191,web,None,None,2023-06-28 17:02:21,2023-06-28 17:02:21,completed,paid,fulfilled,,checkout,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,435690eb-bb34-4801-b6aa-c5584af329ea,Register #100 - samsung SM-T505,e31dfe16-f38a-4d3f-8dca-b5403054567d,Amar,,+6289513933369,,minumyukkaka.adm4@gmail.com,Surya,MYK B

In [13]:
full_iseller_data = proccess_iseller_data(data)

In [14]:
cogs_dict = full_iseller_data["cogs"].to_list()
normalized_cogs = normalize_list_dictionary(cogs_dict)

In [15]:
cogs_df = pd.DataFrame(normalized_cogs)
cogs_df = cogs_df.drop_duplicates(subset=['product_id'], keep='first')

In [16]:
full_iseller_data['order_details'][0]

[{'order_id': 'eff8a50d-8045-4c52-a711-e507f825bc5d',
  'order_detail_id': '4d17af7c-bdd5-4dfc-816e-65e9f38dad56',
  'product_id': 'dd1f826a-2802-4e95-86d7-fad7cbb943ec',
  'product_name': 'Vibe Exotic Lychee 700ml',
  'product_generic_name': 'Vibe Exotic Lychee 700ml',
  'product_type': '',
  'product_variant_id': None,
  'product_variant_name': '',
  'product_modifier_id': None,
  'product_modifier_name': '',
  'order_type': 'pick_up',
  'notes': '',
  'vendor': '',
  'sku': 'LQ27',
  'fulfillment_status': 'fulfilled',
  'quantity': 1.0,
  'base_price': 245000.0,
  'total_order_amount': 245000.0,
  'subtotal': 245000.0,
  'additional_charge': 0.0,
  'discount_amount': 0.0,
  'discount_percentage': None,
  'discount_order_amount': 0.0,
  'taxable': False,
  'tax_percentage': 10.0,
  'tax_amount': 0.0,
  'require_shipping': False,
  'weight': None,
  'weight_quantity': None,
  'type': 'standard',
  'track_inventory': True,
  'modifier_product_type': None,
  'modifier_product_id': None,

In [17]:
cogs_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, 0 to 295
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   product_id  103 non-null    object 
 1   sku         103 non-null    object 
 2   quantity    103 non-null    float64
 3   price       103 non-null    float64
dtypes: float64(2), object(2)
memory usage: 4.0+ KB


In [18]:
full_iseller_data['product_id'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 299 entries, 0 to 298
Series name: product_id
Non-Null Count  Dtype 
--------------  ----- 
297 non-null    object
dtypes: object(1)
memory usage: 2.5+ KB


In [19]:
full_iseller_data['outlet_name'].value_counts()

outlet_name
Bekasi - Perjuangan    299
Name: count, dtype: int64

In [20]:
full_iseller_data['outlet_code'].value_counts()

outlet_code
Bekasi - Perjuangan    299
Name: count, dtype: int64

In [21]:
full_iseller_data.iloc[:, 20:25]

,customer_id,customer_first_name,customer_last_name,customer_phone_number,customer_email
0,a048865d-3c97-465b-8c3d-c54d5f08891e,rafael,,+6281218610542,
1,862b84ac-5523-4d8e-b12f-26e8dd92cd17,azis,,+628383825943554,
2,0c77af02-5608-41cd-af6c-2e3fcdc4da41,rizkya,,+62811107098,
3,0c77af02-5608-41cd-af6c-2e3fcdc4da41,rizkya,,+62811107098,
4,32a9fc73-2ee6-4fb6-8cd6-7389783086fb,michael,,,
...,...,...,...,...,...
294,3f97808d-d031-4ac9-8446-fc217bf72620,Vibe24jam,,,
295,3f97808d-d031-4ac9-8446-fc217bf72620,Vibe24jam,,,
296,3f97808d-d031-4ac9-8446-fc217bf72620,Vibe24jam,,,
297,3f97808d-d031-4ac9-8446-fc217bf72620,Vibe24jam,,,


In [22]:
test = pd.DataFrame(full_iseller_data['cogs'])

In [23]:
full_iseller_data.iloc[69, :]

external_customer_id                                                     
order_id                             1e56016a-3554-42f0-af46-cb6bf0e6ec65
order_reference                                                   #3-1045
channel                                                               pos
channel_detail                                                       None
                                              ...                        
fulfillment_status_y                                            fulfilled
quantity                                                              1.0
base_price                                                       230000.0
total_order_amount                                               230000.0
cogs                    [{'product_id': '61847fd1-3ab6-406a-9604-6ad0d...
Name: 69, Length: 66, dtype: object

In [24]:
full_iseller_data.loc[69]['product_id']
# 'a74b7faa-43cd-435f-87fd-7750a4257042'

'a74b7faa-43cd-435f-87fd-7750a4257042'

In [25]:
full_iseller_data.loc[69]['cogs']

[{'product_id': '61847fd1-3ab6-406a-9604-6ad0de27efa1',
  'sku': 'REG0015',
  'quantity': 1.0,
  'price': 0.0},
 {'product_id': '34fad6f2-9904-4a16-92fa-39d67f97784e',
  'sku': 'WH0034',
  'quantity': 1.0,
  'price': 164800.0}]

In [26]:
full_iseller_data.drop(
    [
        'external_customer_id',
        'device_location',
        'server_last_name',
        'server_first_name',
        'customer_phone_number',
        'customer_email',
        'server_id',
        'customer_last_name',
        'customer_first_name',
        'channel_detail',
        'marketplace_account',
        'order_reference',
        'register_id',
        'register_name',
        'cashier_id',
        'cashier_first_name',
        'cashier_last_name',
        'billing',
        'shipping',
        'processing_method',
        'promotions',
        'discounts',
        'transactions',
    ],
    axis=1,
    inplace=True
)

In [27]:
full_iseller_merge = pd.merge(full_iseller_data, cogs_df, on='product_id', how='left')
full_iseller_merge[full_iseller_merge['price'].isnull()].tail(10)

,order_id,channel,order_date,closed_date,status,payment_status,fulfillment_status_x,notes,cancel_date,cancel_reason,outlet_id,outlet_name,outlet_code,customer_id,currency,grand_total_amount,discount_percentage,total_discount_amount,total_promotion_amount,subtotal,total_tax_amount,total_additional_final_amount,total_additional_subtotal_amount,total_additional_order_amount,rounding_amount,total_amount,buying_price,total_shipping_amount,total_weight,tax_inclusive,on_account,remaining_on_account,order_type,order_details,partner_loyalties,product_id,product_name,product_type,fulfillment_status_y,quantity_x,base_price,total_order_amount,cogs,sku,quantity_y,price
289,c6ead391-9f61-42d6-a9fe-c4bb801c795a,pos,2021-02-26 21:31:20,2021-02-26 21:33:10,completed,paid,fulfilled,ferisetiawan,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,3f97808d-d031-4ac9-8446-fc217bf72620,IDR,504000.0,None,0.0,0.0,504000.0,0.0,0.0,0.0,0.0,0.0,504000.0,428572.0,0.0,0,False,False,0.0,pick_up,[{'order_id': 'c6ead391-9f61-42d6-a9fe-c4bb801...,None,69ddfe4a-2eda-4116-bdb4-71e04df79ed9,V24 - Black tea,,fulfilled,2.0,252000.0,504000.0,[{'product_id': 'ff30df11-5337-46b9-8144-9847e...,NaN,NaN,NaN
290,6c8a639a-af77-4a95-9726-980e71c5da4b,pos,2021-02-26 21:50:20,2021-02-26 21:51:06,completed,paid,fulfilled,reza,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,3f97808d-d031-4ac9-8446-fc217bf72620,IDR,232000.0,None,0.0,0.0,232000.0,0.0,0.0,0.0,0.0,0.0,232000.0,170769.0,0.0,0,False,False,0.0,pick_up,[{'order_id': '6c8a639a-af77-4a95-9726-980e71c...,None,214434d4-651a-4088-bc28-dd78a1d9e9b0,V24 - Cherry Brandy,,fulfilled,1.0,232000.0,232000.0,[{'product_id': '5dc3df26-5bdb-4909-92eb-dbecf...,NaN,NaN,NaN
291,dc38b836-1bd4-499e-91e5-fcc19604bef3,pos,2021-02-26 21:53:42,2021-02-26 21:54:33,completed,paid,fulfilled,dicky,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,3f97808d-d031-4ac9-8446-fc217bf72620,IDR,252000.0,None,0.0,0.0,252000.0,0.0,0.0,0.0,0.0,0.0,252000.0,214286.0,0.0,0,False,False,0.0,pick_up,[{'order_id': 'dc38b836-1bd4-499e-91e5-fcc1960...,None,dd993ede-fc85-483b-861a-d02e3d399f3c,V24 - Exotic Lychee,,fulfilled,1.0,252000.0,252000.0,[{'product_id': 'dd1f826a-2802-4e95-86d7-fad7c...,NaN,NaN,NaN
292,a00a8b76-5eb6-4223-a838-cf9f591ffddc,pos,2021-02-26 21:56:53,2021-02-26 21:58:26,completed,paid,fulfilled,alan,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,3f97808d-d031-4ac9-8446-fc217bf72620,IDR,968000.0,None,0.0,0.0,968000.0,0.0,0.0,0.0,0.0,0.0,968000.0,726593.0,0.0,0,False,False,0.0,pick_up,[{'order_id': 'a00a8b76-5eb6-4223-a838-cf9f591...,None,f8ae0030-c352-4adc-8e01-f40f69d4d791,V24 - Black Tea + Soda Water,,fulfilled,1.0,257000.0,257000.0,[{'product_id': 'ff30df11-5337-46b9-8144-9847e...,NaN,NaN,NaN
293,a00a8b76-5eb6-4223-a838-cf9f591ffddc,pos,2021-02-26 21:56:53,2021-02-26 21:58:26,completed,paid,fulfilled,alan,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,3f97808d-d031-4ac9-8446-fc217bf72620,IDR,968000.0,None,0.0,0.0,968000.0,0.0,0.0,0.0,0.0,0.0,968000.0,726593.0,0.0,0,False,False,0.0,pick_up,[{'order_id': 'a00a8b76-5eb6-4223-a838-cf9f591...,None,040f06ff-8589-411c-986f-9ca3aa061bf8,V24 - Triple Sec + Ginger Ale,,fulfilled,1.0,237000.0,237000.0,[{'product_id': 'f81dc309-6c82-4bde-afed-a5781...,NaN,NaN,NaN
294,a00a8b76-5eb6-4223-a838-cf9f591ffddc,pos,2021-02-26 21:56:53,2021-02-26 21:58:26,completed,paid,fulfilled,alan,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,3f97808d-d031-4ac9-8446-fc217bf72620,IDR,968000.0,None,0.0,0.0,968000.0,0.0,0.0,0.0,0.0,0.0,968000.0,726593.0,0.0,0,False,False,0.0,pick_up,[{'order_id': 'a00a8b76-5eb6-4223-a838-cf9f591...,None,e76d299e-30d7-41a5-8ab1-273f31c83977,V24 - Light Rum + Ginger Ale,,fulfilled,2.0,237000.0,474000.0,[{'product_id': '9f5dae85-99e5-45b0-8cb2-0ea27...,NaN,NaN,NaN
295,a432b68a-c0f1-426a-89f1-04c8

In [28]:
full_iseller_data[full_iseller_merge['order_id'] == 'a00a8b76-5eb6-4223-a838-cf9f591ffddc']

,order_id,channel,order_date,closed_date,status,payment_status,fulfillment_status_x,notes,cancel_date,cancel_reason,outlet_id,outlet_name,outlet_code,customer_id,currency,grand_total_amount,discount_percentage,total_discount_amount,total_promotion_amount,subtotal,total_tax_amount,total_additional_final_amount,total_additional_subtotal_amount,total_additional_order_amount,rounding_amount,total_amount,buying_price,total_shipping_amount,total_weight,tax_inclusive,on_account,remaining_on_account,order_type,order_details,partner_loyalties,product_id,product_name,product_type,fulfillment_status_y,quantity,base_price,total_order_amount,cogs
292,a00a8b76-5eb6-4223-a838-cf9f591ffddc,pos,2021-02-26 21:56:53,2021-02-26 21:58:26,completed,paid,fulfilled,alan,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,3f97808d-d031-4ac9-8446-fc217bf72620,IDR,968000.0,None,0.0,0.0,968000.0,0.0,0.0,0.0,0.0,0.0,968000.0,726593.0,0.0,0,False,False,0.0,pick_up,[{'order_id': 'a00a8b76-5eb6-4223-a838-cf9f591...,None,f8ae0030-c352-4adc-8e01-f40f69d4d791,V24 - Black Tea + Soda Water,,fulfilled,1.0,257000.0,257000.0,[{'product_id': 'ff30df11-5337-46b9-8144-9847e...
293,a00a8b76-5eb6-4223-a838-cf9f591ffddc,pos,2021-02-26 21:56:53,2021-02-26 21:58:26,completed,paid,fulfilled,alan,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,3f97808d-d031-4ac9-8446-fc217bf72620,IDR,968000.0,None,0.0,0.0,968000.0,0.0,0.0,0.0,0.0,0.0,968000.0,726593.0,0.0,0,False,False,0.0,pick_up,[{'order_id': 'a00a8b76-5eb6-4223-a838-cf9f591...,None,040f06ff-8589-411c-986f-9ca3aa061bf8,V24 - Triple Sec + Ginger Ale,,fulfilled,1.0,237000.0,237000.0,[{'product_id': 'f81dc309-6c82-4bde-afed-a5781...
294,a00a8b76-5eb6-4223-a838-cf9f591ffddc,pos,2021-02-26 21:56:53,2021-02-26 21:58:26,completed,paid,fulfilled,alan,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,3f97808d-d031-4ac9-8446-fc217bf72620,IDR,968000.0,None,0.0,0.0,968000.0,0.0,0.0,0.0,0.0,0.0,968000.0,726593.0,0.0,0,False,False,0.0,pick_up,[{'order_id': 'a00a8b76-5eb6-4223-a838-cf9f591...,None,e76d299e-30d7-41a5-8ab1-273f31c83977,V24 - Light Rum + Ginger Ale,,fulfilled,2.0,237000.0,474000.0,[{'product_id': '9f5dae85-99e5-45b0-8cb2-0ea27...


In [29]:
df[df['product_name'].str.startswith('cabang -', na=True)]

,external_customer_id,order_id,order_reference,channel,channel_detail,marketplace_account,order_date,closed_date,status,payment_status,fulfillment_status_x,notes,processing_method,cancel_date,cancel_reason,outlet_id,outlet_name,outlet_code,register_id,register_name,customer_id,customer_first_name,customer_last_name,customer_phone_number,customer_email,server_id,server_first_name,server_last_name,cashier_id,cashier_first_name,cashier_last_name,currency,grand_total_amount,discount_percentage,total_discount_amount,total_promotion_amount,subtotal,total_tax_amount,total_additional_final_amount,total_additional_subtotal_amount,total_additional_order_amount,rounding_amount,total_amount,buying_price,total_shipping_amount,total_weight,tax_inclusive,on_account,remaining_on_account,billing,shipping,device_location,order_type,order_details,promotions,discounts,transactions,partner_loyalties,product_id,product_name,product_type,fulfillment_status_y,quantity,base_price,total_order_amount,cogs
99,None,60a7431d-74e2-4549-a996-fe6329211041,2312122DHSE24D,marketplace,shopee,Shopee - Minum Yukkaka Official Store,2023-12-12 12:27:55,2023-12-12 12:27:55,void,refunded,none,,checkout,2023-12-12 12:28:50,customer,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,None,None,None,None,None,None,None,None,None,None,None,None,None,IDR,554000.0,None,0.0,18000.0,536000.0,0.0,1000.0,0.0,0.0,0.0,555000.0,0.0,18000.0,13,False,False,0.0,None,"{'address_id': None, 'contact_title': None, 'f...",None,,[],[],[],[],None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,None,d45eb876-b288-4509-a20c-e7359f8cbf04,#193118,web,None,None,2023-12-12 13:01:36,2023-12-12 13:01:36,completed,paid,none,,checkout,None,,4accd2a8-ddd2-0327-10e6-94ef5cec3b84,Jaksel - Pondok Indah,Jaksel - Pondok Indah,bf823f57-df43-4101-b64b-6b3684b4d691,Register #96 - samsung SM-X205,e6ba98fe-e5f9-4f00-b9f9-aa57b83d7144,Nabilla,,+6281283170994,None,frengkyando.domo@gmail.com,Ando,MYK Jaksel,None,None,None,IDR,290000.0,None,0.0,0.0,290000.0,0.0,0.0,0.0,0.0,0.0,316214.0,0.0,24000.0,3880,False,False,0.0,None,{'address_id': 'f209ee3f-ece8-427d-b645-b184b5...,None,delivery,[],[],[],[],None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
full_iseller_merge.tail()

,order_id,channel,order_date,closed_date,status,payment_status,fulfillment_status_x,notes,cancel_date,cancel_reason,outlet_id,outlet_name,outlet_code,customer_id,currency,grand_total_amount,discount_percentage,total_discount_amount,total_promotion_amount,subtotal,total_tax_amount,total_additional_final_amount,total_additional_subtotal_amount,total_additional_order_amount,rounding_amount,total_amount,buying_price,total_shipping_amount,total_weight,tax_inclusive,on_account,remaining_on_account,order_type,order_details,partner_loyalties,product_id,product_name,product_type,fulfillment_status_y,quantity_x,base_price,total_order_amount,cogs,sku,quantity_y,price
294,a00a8b76-5eb6-4223-a838-cf9f591ffddc,pos,2021-02-26 21:56:53,2021-02-26 21:58:26,completed,paid,fulfilled,alan,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,3f97808d-d031-4ac9-8446-fc217bf72620,IDR,968000.0,None,0.0,0.0,968000.0,0.0,0.0,0.0,0.0,0.0,968000.0,726593.0,0.0,0,False,False,0.0,pick_up,[{'order_id': 'a00a8b76-5eb6-4223-a838-cf9f591...,None,e76d299e-30d7-41a5-8ab1-273f31c83977,V24 - Light Rum + Ginger Ale,,fulfilled,2.0,237000.0,474000.0,[{'product_id': '9f5dae85-99e5-45b0-8cb2-0ea27...,NaN,NaN,NaN
295,a432b68a-c0f1-426a-89f1-04c86ec06ff1,pos,2021-02-26 21:59:10,2021-02-26 22:00:06,completed,paid,fulfilled,horrison,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,3f97808d-d031-4ac9-8446-fc217bf72620,IDR,252000.0,None,0.0,0.0,252000.0,0.0,0.0,0.0,0.0,0.0,252000.0,214286.0,0.0,0,False,False,0.0,pick_up,[{'order_id': 'a432b68a-c0f1-426a-89f1-04c86ec...,None,dd993ede-fc85-483b-861a-d02e3d399f3c,V24 - Exotic Lychee,,fulfilled,1.0,252000.0,252000.0,[{'product_id': 'dd1f826a-2802-4e95-86d7-fad7c...,NaN,NaN,NaN
296,e8f43db1-9bdc-42ec-8b31-7cd1abd5e642,pos,2021-02-26 22:00:55,2021-02-26 22:01:28,completed,paid,fulfilled,inal,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,3f97808d-d031-4ac9-8446-fc217bf72620,IDR,232000.0,None,0.0,0.0,232000.0,0.0,0.0,0.0,0.0,0.0,232000.0,170769.0,0.0,0,False,False,0.0,pick_up,[{'order_id': 'e8f43db1-9bdc-42ec-8b31-7cd1abd...,None,768b41a2-9feb-4585-b389-d9631562464d,V24 - Triple Sec,,fulfilled,1.0,232000.0,232000.0,[{'product_id': 'f81dc309-6c82-4bde-afed-a5781...,NaN,NaN,NaN
297,8f0b43c1-c19c-4318-82f0-9819275f14fc,pos,2021-02-26 22:05:11,2021-02-26 22:05:56,completed,paid,fulfilled,dadan,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,3f97808d-d031-4ac9-8446-fc217bf72620,IDR,257000.0,None,0.0,0.0,257000.0,0.0,0.0,0.0,0.0,0.0,257000.0,9416.0,0.0,0,False,False,0.0,pick_up,[{'order_id': '8f0b43c1-c19c-4318-82f0-9819275...,None,e4290fb6-aaf5-42ac-9cc5-964a635215df,V24 - Peach + Tonic Water,,fulfilled,1.0,257000.0,257000.0,[{'product_id': 'a4fab552-3fe4-4acd-b9fa-32dcd...,NaN,NaN,NaN
298,53df951d-c860-42ab-971c-f82db5a20856,pos,2021-02-26 22:06:33,2021-02-26 22:07:19,completed,paid,fulfilled,anjas,None,,a84333eb-f1ae-4ffc-ba80-72b2c5ed7e14,Bekasi - Perjuangan,Bekasi - Perjuangan,3f97808d-d031-4ac9-8446-fc217bf72620,IDR,252000.0,None,0.0,0.0,252000.0,0.0,0.0,0.0,0.0,0.0,252000.0,214286.0,0.0,0,False,False,0.0,pick_up,[{'order_id': '53df951d-c860-42ab-971c-f82db5a...,None,69ddfe4a-2eda-4116-bdb4-71e04df79ed9,V24 - Black tea,,fulfilled,1.0,252000.0,252000.0,[{'product_id': 'ff30df11-5337-46b9-8144-9847e...,NaN,NaN,NaN
